In [ ]:
import pandas as pd
import numpy as np

# Load your CSV file (update path if needed)
df = pd.read_csv("/Users/rabkhan/Desktop/Projects/assignment/ecom_data.csv")

print(df.head())

   user_id Attribute-1 Attribute-2 Attribute-3 Attribute-4 Attribute-5  \
0  7573670          a1          b2          c6          d1          e9   
1  7573671          a2         NaN         NaN         NaN         NaN   
2  7573672          a2          b2          c2         NaN         NaN   
3  7573673          a1         NaN         NaN         NaN         NaN   
4  7573674          a1         NaN         NaN         NaN         NaN   

   week1gmv  week2gmv  week3gmv  week4gmv  week5gmv  week6gmv  week7gmv  \
0         0         0         0         0         0         0         0   
1         0         0         0         0         0         0         0   
2         0         0         0         0         0         0         0   
3         0         0         0         0         0         0         0   
4         0         0         0         0         0         0         0   

   week8gmv  week9gmv  week10gmv  week11gmv  week12gmv  
0         0         0          0          0    

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333281 entries, 0 to 333280
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      333281 non-null  int64 
 1   Attribute-1  333281 non-null  object
 2   Attribute-2  237376 non-null  object
 3   Attribute-3  237281 non-null  object
 4   Attribute-4  257493 non-null  object
 5   Attribute-5  253530 non-null  object
 6   week1gmv     333281 non-null  int64 
 7   week2gmv     333281 non-null  int64 
 8   week3gmv     333281 non-null  int64 
 9   week4gmv     333281 non-null  int64 
 10  week5gmv     333281 non-null  int64 
 11  week6gmv     333281 non-null  int64 
 12  week7gmv     333281 non-null  int64 
 13  week8gmv     333281 non-null  int64 
 14  week9gmv     333281 non-null  int64 
 15  week10gmv    333281 non-null  int64 
 16  week11gmv    333281 non-null  int64 
 17  week12gmv    333281 non-null  int64 
dtypes: int64(13), object(5)
memory usage: 45.8+ 

In [18]:
# Making sure all GMV week columns are numeric
for col in week_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Checking if there are any non-numeric values
print(df[week_cols].dtypes)


week1gmv     int64
week2gmv     int64
week3gmv     int64
week4gmv     int64
week5gmv     int64
week6gmv     int64
week7gmv     int64
week8gmv     int64
week9gmv     int64
week10gmv    int64
week11gmv    int64
week12gmv    int64
dtype: object


In [14]:
# Test: how many users have any GMV > 0
test_active = (df[week_cols].sum(axis=1) > 0).sum()
print("Users with >0 GMV:", test_active)


Users with >0 GMV: 32722


In [15]:
# Identifying week columns and creating activation flag

# Select all columns that represent weekly GMV
week_cols = [col for col in df.columns if col.lower().startswith("week") and "gmv" in col.lower()]

week_cols = sorted(week_cols, key=lambda x: int(''.join(filter(str.isdigit, x)) or 0))  # ensure week1..week12 order
print("Detected weekly GMV columns:", week_cols)

# Create an 'activated' flag: 1 if user has >0 GMV in any week, else 0
df["activated"] = (df[week_cols].sum(axis=1) > 0).astype(int)

# Check basic activation statistics
total_users = len(df)
activated_users = df["activated"].sum()
activation_rate = activated_users / total_users

print(f"\nTotal users: {total_users}")
print(f"Activated users: {activated_users}")
print(f"Overall activation rate: {activation_rate:.4%}")

# Optional: Quick sanity check (see a few rows)
print("\nSample data with activation flag:")
print(df[["user_id", "activated"] + week_cols[:3]].head(10))


Detected weekly GMV columns: ['week1gmv', 'week2gmv', 'week3gmv', 'week4gmv', 'week5gmv', 'week6gmv', 'week7gmv', 'week8gmv', 'week9gmv', 'week10gmv', 'week11gmv', 'week12gmv']

Total users: 333281
Activated users: 32722
Overall activation rate: 9.8181%

Sample data with activation flag:
   user_id  activated  week1gmv  week2gmv  week3gmv
0  7573670          0         0         0         0
1  7573671          0         0         0         0
2  7573672          0         0         0         0
3  7573673          0         0         0         0
4  7573674          0         0         0         0
5  7573675          0         0         0         0
6  7573676          0         0         0         0
7  7573677          1      4223      3670      5121
8  7573678          0         0         0         0
9  7573679          0         0         0         0


In [16]:
# Find the week columns again
week_cols = [c for c in df.columns if c.lower().startswith("week") and "gmv" in c.lower()]
week_cols

df[week_cols].head()


,week1gmv,week2gmv,week3gmv,week4gmv,week5gmv,week6gmv,week7gmv,week8gmv,week9gmv,week10gmv,week11gmv,week12gmv
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Create a list (or boolean array) to see which week had any order
active_weeks = df[week_cols] > 0

# Find the first week where the user made a purchase
first_week = []

for i in range(len(df)):
    weeks = active_weeks.iloc[i].values
    if True in weeks:
        first_week_num = list(weeks).index(True) + 1   # +1 because weeks start from 1
    else:
        first_week_num = None
    first_week.append(first_week_num)

df["first_act_week"] = first_week

df[["user_id", "first_act_week"]].head(10)


,user_id,first_act_week
0,7573670,NaN
1,7573671,NaN
2,7573672,NaN
3,7573673,NaN
4,7573674,NaN
5,7573675,NaN
6,7573676,NaN
7,7573677,1.0
8,7573678,NaN
9,7573679,NaN


In [23]:
Wa = []   # number of active weeks after activation
Wn = []   # number of inactive weeks after activation
ta = []   # total number of weeks after activation till week12

for i in range(len(df)):
    first = df.loc[i, "first_act_week"]

    # if not activated, just add None values
    if pd.isna(first):
        Wa.append(None)
        Wn.append(None)
        ta.append(None)
    else:
        # Take from activation week till week12 
        user_weeks = df.loc[i, week_cols[int(first) - 1 :]]
        active_count = sum(user_weeks > 0)
        total_weeks = len(user_weeks)
        non_active_count = total_weeks - active_count

        Wa.append(active_count)
        Wn.append(non_active_count)
        ta.append(total_weeks)

df["Wa"] = Wa
df["Wn"] = Wn
df["ta"] = ta

df[["user_id", "first_act_week", "Wa", "Wn", "ta"]].head(10)



,user_id,first_act_week,Wa,Wn,ta
0,7573670,NaN,NaN,NaN,NaN
1,7573671,NaN,NaN,NaN,NaN
2,7573672,NaN,NaN,NaN,NaN
3,7573673,NaN,NaN,NaN,NaN
4,7573674,NaN,NaN,NaN,NaN
5,7573675,NaN,NaN,NaN,NaN
6,7573676,NaN,NaN,NaN,NaN
7,7573677,1.0,4.0,8.0,12.0
8,7573678,NaN,NaN,NaN,NaN
9,7573679,NaN,NaN,NaN,NaN


In [25]:
# Calculating retention score for each user

# set penalty value j
j = 1

Rs = []  # will store the score for each user

for i in range(len(df)):
    if pd.isna(df.loc[i, "Wa"]):   # means not activated
        Rs.append(None)
    else:
        Wa = df.loc[i, "Wa"]
        Wn = df.loc[i, "Wn"]
        ta = df.loc[i, "ta"]

        score = ((Wa - j * Wn) * 100) / ta
        Rs.append(score)

df["Rs"] = Rs

df[["user_id", "Wa", "Wn", "ta", "Rs"]].head(10)


,user_id,Wa,Wn,ta,Rs
0,7573670,NaN,NaN,NaN,NaN
1,7573671,NaN,NaN,NaN,NaN
2,7573672,NaN,NaN,NaN,NaN
3,7573673,NaN,NaN,NaN,NaN
4,7573674,NaN,NaN,NaN,NaN
5,7573675,NaN,NaN,NaN,NaN
6,7573676,NaN,NaN,NaN,NaN
7,7573677,4.0,8.0,12.0,-33.333333
8,7573678,NaN,NaN,NaN,NaN
9,7573679,NaN,NaN,NaN,NaN


In [26]:
# checking how many activated users have valid Rs
print("Users with retention score:", df["Rs"].notna().sum())

# check summary stats
print("\nRetention score summary (activated users only):")
print(df.loc[df["Rs"].notna(), "Rs"].describe())

Users with retention score: 32722

Retention score summary (activated users only):
count    32722.000000
mean       -49.758107
std         47.157177
min        -83.333333
25%        -83.333333
50%        -66.666667
75%        -33.333333
max        100.000000
Name: Rs, dtype: float64


In [27]:
# identify all attribute columns
attribute_cols = [c for c in df.columns if c.lower().startswith("attribute")]
attribute_cols


['Attribute-1', 'Attribute-2', 'Attribute-3', 'Attribute-4', 'Attribute-5']

In [28]:
# Activation probability for each attribute value
for col in attribute_cols:
    temp = df.groupby(col).agg(
        total_users=('user_id', 'count'),
        activated_users=('activated', 'sum')
    )
    temp["activation_rate"] = (temp["activated_users"] / temp["total_users"]) * 100

    print(f"\nTop values by activation rate for {col}:")
    print(temp.sort_values("activation_rate", ascending=False).head(5))



Top values by activation rate for Attribute-1:
             total_users  activated_users  activation_rate
Attribute-1                                               
a9                   846              808        95.508274
a3                 10366             3341        32.230369
a10                  847              116        13.695396
a2                 83013            10622        12.795586
a7                 14184             1512        10.659898

Top values by activation rate for Attribute-2:
             total_users  activated_users  activation_rate
Attribute-2                                               
b1                 75610            14937        19.755323
b2                161766            15466         9.560724

Top values by activation rate for Attribute-3:
             total_users  activated_users  activation_rate
Attribute-3                                               
c1                 44217             8526        19.282177
c4                  9987      

In [29]:
# only activated users for retention analysis
activated_users = df[df["activated"] == 1]

for col in attribute_cols:
    temp = activated_users.groupby(col).agg(
        total_activated=('user_id', 'count'),
        avg_retention=('Rs', 'mean')
    )
    print(f"\nTop values by average retention for {col}:")
    print(temp.sort_values("avg_retention", ascending=False).head(5))



Top values by average retention for Attribute-1:
             total_activated  avg_retention
Attribute-1                                
a5                       472     -46.647269
a10                      116     -46.760648
a12                      179     -46.792949
a3                      3341     -47.174240
a15                       21     -47.313269

Top values by average retention for Attribute-2:
             total_activated  avg_retention
Attribute-2                                
b1                     14937     -36.046958
b2                     15466     -61.303960

Top values by average retention for Attribute-3:
             total_activated  avg_retention
Attribute-3                                
c1                      8526     -33.601305
c4                      1460     -44.415169
c2                      2616     -51.883202
c3                      1680     -55.037406
c5                      9234     -55.612987

Top values by average retention for Attribute-4:
        

<H3>Activation Summary </H3>

<p>Attribute-1
a9 (95.5 %), a3 (32 %)
Mean - a9 users are almost guaranteed to activate

Attribute-2	
b1 (19.7 %), b2(9.5 %)	| b1 is roughly twice as likely to activate as b2

Attribute-3	
c1 (19.3 %) > c4 (14.6 %) | clear differences in user behavior across sub-segments

Overall activation rate	9.8 % - baseline for comparison
</p>

<h3>Retention Summary (Average Rs)</h3>

<h4>Attribute-1</h4>
<table>
<tr><th>Value</th><th>Activated Users</th><th>Avg Retention (Rs)</th></tr>
<tr><td>a5</td><td>472</td><td>-46.65</td></tr>
<tr><td>a10</td><td>116</td><td>-46.76</td></tr>
<tr><td>a12</td><td>179</td><td>-46.79</td></tr>
<tr><td>a3</td><td>3341</td><td>-47.17</td></tr>
<tr><td>a15</td><td>21</td><td>-47.31</td></tr>
</table>
<p>a5 users retain slightly better than others under Attribute-1.</p>

<h4>Attribute-2</h4>
<table>
<tr><th>Value</th><th>Activated Users</th><th>Avg Retention (Rs)</th></tr>
<tr><td>b1</td><td>14,937</td><td>-36.05</td></tr>
<tr><td>b2</td><td>15,466</td><td>-61.30</td></tr>
</table>
<p>b1 users show much better retention than b2. They stay active longer and purchase more weeks after activation.</p>

<h4>Attribute-3</h4>
<table>
<tr><th>Value</th><th>Activated Users</th><th>Avg Retention (Rs)</th></tr>
<tr><td>c1</td><td>8,526</td><td>-33.60</td></tr>
<tr><td>c4</td><td>1,460</td><td>-44.42</td></tr>
<tr><td>c2</td><td>2,616</td><td>-51.88</td></tr>
<tr><td>c3</td><td>1,680</td><td>-55.04</td></tr>
<tr><td>c5</td><td>9,234</td><td>-55.61</td></tr>
</table>
<p>c1 users have the best retention scores overall. They continue buying more often compared to other segments like c3 or c5.</p>

<h4>Other Attribute Trends</h4>
<table>
<tr><th>Attribute</th><th>Top Values</th><th>Avg Retention (approx)</th><th>Notes</th></tr>
<tr><td>Attribute-5</td><td>e3, e6, e18, e22</td><td>-42 to -46</td><td>e3 users are slightly more consistent in repeat activity.</td></tr>
</table>

<hr>

<h3>What This Means</h3>

<ul>
<li><b>High activation + high retention:</b> segments like <b>a9 + b1 + c1</b> are your best performing group. These users adopt fast and keep coming back.</li>
<li><b>Low activation or low retention:</b> groups like <b>b2, c5</b> may activate but churn quickly. They could benefit from better onboarding or post-purchase nudges.</li>
<li><b>Overall:</b> retention scores are negative because users usually have more inactive weeks than active ones — but the <i>relative differences</i> still tell us who stays longer.</li>
</ul>


<h3>Actionable Insights (as PM)</h3>

<ol>
<li><b>Acquire more of the right users:</b> focus marketing on profiles similar to <b>a9</b>, <b>b1</b>, and <b>c1</b>. These users activate and retain best.</li>
<li><b>Fix early churn:</b> investigate journeys for <b>b2</b> and <b>c5</b> users. Improve onboarding or send “win-back” offers after week 2-3.</li>
<li><b>Personalize engagement:</b> 
   <ul>
      <li>Give loyal <b>b1/c1</b> users loyalty rewards or special offers.</li>
      <li>Re-target <b>a3</b> or <b>c4</b> users with gentle reminders or discounts.</li>
   </ul>
</li>
<li><b>Goal for next quarter:</b> raise activation from <b>9.8 % → 12 %+</b> and improve average retention (Rs) from <b>-49 → -35</b>.</li>
</ol>



<h3>In Simple Terms</h3>

<p>
The platform has plenty of sign-ups, but only about 1 in 10 users actually order.<br>
Among those who do, a few clear groups, especially <b>a9</b>, <b>b1</b>, and <b>c1</b> — drive most repeat purchases.<br>
If we attract more of those users and help weaker ones like <b>b2</b> stay longer, both activation and retention will grow steadily.
</p>